In [1]:
from functions import *

In [2]:
print(tf.test.gpu_device_name())
print(tf.__version__)
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)

/device:GPU:0
2.3.0


## Load and re-organize data

In [3]:
dataset_path = 'data/00a-segments_normWithinSubjRunROI.pkl'
with open(dataset_path, 'rb') as f:
    dataset = pkl.load(f)

print("Using dataset: {0}".format(dataset_path))
num_subjs = len(dataset.keys())
print("There are {num_subjs} participants in the dataset".format(num_subjs=num_subjs))

p1 = list(dataset.keys())[0]
p1_data = dataset[p1]['data']
print("Each participant's data is a {0}D array of shape {1}".format(len(p1_data.shape),
                                                                   p1_data.shape))

dataset_df = organize_dataset(dataset)

Using dataset: data/00a-segments_normWithinSubjRunROI.pkl
There are 61 participants in the dataset
Each participant's data is a 3D array of shape (14, 316, 39)


## Train-test split

In [4]:
rs = ShuffleSplit(n_splits=50, train_size=.7, random_state=0)
all_participants = dataset_df.participant.unique()

batch_size = 32

for nonTest_idx, test_idx in rs.split(all_participants):
    pass

## Cross validation

In [1]:
#params = {'l2': 0, 'dropout': 0.4, 'kernel_size': 2, 'lr': 0.01, 'strides': 1} # worked well for rAI
#params = {'l2': 1, 'dropout': 0.4, 'kernel_size': 2, 'lr': 0.001, 'strides': 1} # works better for this dataset if using top conv layer
params = {'l2': 0.001, 'dropout': 0.4, 'kernel_size': 2, 'lr': 0.001, 'strides': 1}
results, results_temp, _ = cross_val_scores(dataset_df,nonTest_idx, params, temp_acc=True, k_perm=0)

## Test accuracy

In [6]:
train_X, train_Y = query_dataset(dataset_df, nonTest_idx)
test_X, test_Y = query_dataset(dataset_df, test_idx)
model = classifier(train_X,train_Y,
                   params['l2'],
                   params['dropout'],
                   params['kernel_size'],
                   params['strides'],
                   params['lr'])
train_acc = model.evaluate(train_X,train_Y)[1]
test_acc = model.evaluate(test_X,test_Y)[1]
print('Train Accuracy: ',train_acc)
print('Test Accuracy: ',test_acc)

54/54 [==============================] - 1s 18ms/step - loss: 0.3038 - acc: 0.9017
Train Accuracy:  0.9449517726898193
Test Accuracy:  0.9017441868782043


## Permutation

In [7]:
results_rand = permute(train_X, train_Y, test_X, test_Y, params, k_perm=100)

--------perm = 0--------
54/54 [==============================] - 1s 13ms/step - loss: 0.7956 - acc: 0.4785
permuted train acc: 0.691
permuted val acc: 0.478
54/54 [==============================] - 1s 12ms/step - loss: 0.8662 - acc: 0.4640


In [9]:
with open('results/accuracy_cv_perm_ROI316.pkl','wb') as f:
    pkl.dump([results, 
              results_temp,
              results_rand,
              {'train':train_acc,'test':test_acc}],f)
    
model.save("models/CustomGRU_ROI316.h5")
# model = tf.keras.models.load_model("models/CustomGRU_ROI316.h5") to load the model.